# Inference notenook for [CLIP prefix captioning](https://github.com/rmokady/CLIP_prefix_caption/) on multiple images

Disclaimer: the authors do not own any rights for the code or data.

In [ ]:
#@title Install
!pip install transformers
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-592zv0t5
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-592zv0t5


In [ ]:
#@title Clone CLIP
import os
!git clone https://github.com/hila-chefer/Transformer-MM-Explainability

!git clone https://github.com/matthewmurakami/CS1430-MapCap.git

!unzip CS1430-MapCap/MapCap_partition.zip 

os.chdir(f'./Transformer-MM-Explainability')

!pip install einops
!pip install ftfy
!pip install captum

Cloning into 'Transformer-MM-Explainability'...
remote: Enumerating objects: 1471, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 1471 (delta 79), reused 58 (delta 54), pack-reused 1373
Receiving objects: 100% (1471/1471), 29.01 MiB | 33.64 MiB/s, done.
Resolving deltas: 100% (568/568), done.
Cloning into 'CS1430-MapCap'...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (200/200), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 458 (delta 64), reused 194 (delta 61), pack-reused 258
Receiving objects: 100% (458/458), 7.27 MiB | 12.30 MiB/s, done.
Resolving deltas: 100% (130/130), done.
Archive:  CS1430-MapCap/MapCap_partition.zip
   creating: MapCap_partition/
   creating: MapCap_partition/apron/
   creating: MapCap_partition/tarantula/
   creating: MapCap_partition/bell pepper/
   creating: MapCap_partition/Egyptian cat/
   creating: MapCap_partition/plunger/
   creating: 

In [ ]:
#@title Drive Downloader

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

download_with_pydrive = True #@param {type:"boolean"}  

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_id, file_dst):
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id $file_id -O $file_dst

downloader = Downloader(download_with_pydrive)

In [ ]:
#@title If error from .ipynb_checkpoints

#os.rmdir("../MapCap_partition/.ipynb_checkpoints")

In [ ]:
#@title Run CLIP (Only pick one of the checkbox options)
import torchvision
import torch
import CLIP.clip as clip
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from captum.attr import visualization
from CLIP.clip.simple_tokenizer import SimpleTokenizer
from skimage.filters import sobel
from skimage import segmentation, transform
from scipy import ndimage as ndi
import shutil


start_layer =  -1#@param {type:"number"}
start_layer_text =  -1#@param {type:"number"}
weighted_darkness = False #@param {type:"boolean"}

def interpret(image, texts, model, device, start_layer=start_layer, start_layer_text=start_layer_text):
    batch_size = texts.shape[0]
    images = image.repeat(batch_size, 1, 1, 1)
    logits_per_image, logits_per_text = model(images, texts)
    probs = logits_per_image.softmax(dim=-1).detach().cpu().numpy()
    index = [i for i in range(batch_size)]
    one_hot = np.zeros((logits_per_image.shape[0], logits_per_image.shape[1]), dtype=np.float32)
    one_hot[torch.arange(logits_per_image.shape[0]), index] = 1
    one_hot = torch.from_numpy(one_hot).requires_grad_(True)
    one_hot = torch.sum(one_hot.cuda() * logits_per_image)
    model.zero_grad()

    image_attn_blocks = list(dict(model.visual.transformer.resblocks.named_children()).values())

    if start_layer == -1: 
      # calculate index of last layer 
      start_layer = len(image_attn_blocks) - 1
    
    num_tokens = image_attn_blocks[0].attn_probs.shape[-1]
    R = torch.eye(num_tokens, num_tokens, dtype=image_attn_blocks[0].attn_probs.dtype).to(device)
    R = R.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
    for i, blk in enumerate(image_attn_blocks):
        if i < start_layer:
          continue
        grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        cam = blk.attn_probs.detach()
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        cam = grad * cam
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        cam = cam.clamp(min=0).mean(dim=1)
        R = R + torch.bmm(cam, R)
    image_relevance = R[:, 0, 1:]

    
    text_attn_blocks = list(dict(model.transformer.resblocks.named_children()).values())

    if start_layer_text == -1: 
      # calculate index of last layer 
      start_layer_text = len(text_attn_blocks) - 1

    num_tokens = text_attn_blocks[0].attn_probs.shape[-1]
    R_text = torch.eye(num_tokens, num_tokens, dtype=text_attn_blocks[0].attn_probs.dtype).to(device)
    R_text = R_text.unsqueeze(0).expand(batch_size, num_tokens, num_tokens)
    for i, blk in enumerate(text_attn_blocks):
        if i < start_layer_text:
          continue
        grad = torch.autograd.grad(one_hot, [blk.attn_probs], retain_graph=True)[0].detach()
        cam = blk.attn_probs.detach()
        cam = cam.reshape(-1, cam.shape[-1], cam.shape[-1])
        grad = grad.reshape(-1, grad.shape[-1], grad.shape[-1])
        cam = grad * cam
        cam = cam.reshape(batch_size, -1, cam.shape[-1], cam.shape[-1])
        cam = cam.clamp(min=0).mean(dim=1)
        R_text = R_text + torch.bmm(cam, R_text)
    text_relevance = R_text
   
    return text_relevance, image_relevance

def show_image_relevance(image_relevance, image, orig_image):
    # create heatmap from mask on image
    def show_cam_on_image(img, mask):
        heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
        heatmap = np.float32(heatmap) / 255
        cam = heatmap + np.float32(img)
        cam = cam / np.max(cam)
        return cam

    fig, axs = plt.subplots(1, 2)
    axs[0].imshow(orig_image);
    axs[0].axis('off');

    dim = int(image_relevance.numel() ** 0.5)
    image_relevance = image_relevance.reshape(1, 1, dim, dim)
    image_relevance = torch.nn.functional.interpolate(image_relevance, size=224, mode='bilinear')
    image_relevance = image_relevance.reshape(224, 224).cuda().data.cpu().numpy()
    image_relevance = (image_relevance - image_relevance.min()) / (image_relevance.max() - image_relevance.min())
    image = image[0].permute(1, 2, 0).data.cpu().numpy()
    image = (image - image.min()) / (image.max() - image.min())
    vis = show_cam_on_image(image, image_relevance)
    vis = np.uint8(255 * vis)
    vis = cv2.cvtColor(np.array(vis), cv2.COLOR_RGB2BGR)
    axs[1].imshow(vis);
    axs[1].axis('off');

_tokenizer = clip.simple_tokenizer.SimpleTokenizer()

def show_heatmap_on_text(text, text_encoding, R_text):
  CLS_idx = text_encoding.argmax(dim=-1)
  R_text = R_text[CLS_idx, 1:CLS_idx]
  text_scores = R_text / R_text.sum()
  text_scores = text_scores.flatten()
  print(text_scores)
  text_tokens=_tokenizer.encode(text)
  text_tokens_decoded=[_tokenizer.decode([a]) for a in text_tokens]
  vis_data_records = [visualization.VisualizationDataRecord(text_scores,0,0,0,0,0,text_tokens_decoded,1)]
  visualization.visualize_text(vis_data_records)

clip.clip._MODELS = {
    "ViT-B/32": "https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt",
    "ViT-B/16": "https://openaipublic.azureedge.net/clip/models/5806e77cd80f8b59890b7e101eabd078d9fb84e6937f9e85e4ecb61988df416f/ViT-B-16.pt",
    "ViT-L/14": "https://openaipublic.azureedge.net/clip/models/b8cca3fd41ae0c99ba7e8951adf17d267cdb84cd88be6f7c2e0eca1737a03836/ViT-L-14.pt",
}

device = "cuda" if torch.cuda.is_available() else "cpu"

model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def segment_heatmap(heatmap):
  dim = int(heatmap.numel() ** 0.5)
  heatmap = heatmap.reshape(1, 1, dim, dim)
  heatmap = torch.nn.functional.interpolate(heatmap, size=224, mode='bilinear')
  heatmap = heatmap.reshape(1, 224, 224).to(device).data.cpu().numpy()
  heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())

  segments = []

  threshold = .5
  mask = np.where(heatmap >= threshold, 2, 0)

  heatmap_mask_no_segmenting = False #@param {type:"boolean"}

  if heatmap_mask_no_segmenting:
    segments.append( (mask, threshold) )
    return segments
  elif weighted_darkness:
    segments.append( (heatmap, 0) )
    return segments

  while True:
      one_elements = np.argwhere(mask == 2)
      seed_pixel = one_elements[0] if len(one_elements) != 0 else []

      if len(seed_pixel) == 0:
          break

      mask = segmentation.flood_fill(mask, tuple(seed_pixel), 1)
      curr_mask = mask == 1

      seg_to_crop = heatmap * curr_mask

      segments.append((seg_to_crop, threshold))
      mask = mask - (2 * curr_mask) # Get rid of segment you just extracted


  return segments
  

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

device = 'cuda'
    

model, preprocess = clip.load("ViT-B/32", device=device, jit=False)


transform = torchvision.transforms.Compose([torchvision.transforms.Resize(224), torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.ImageFolder("/content/MapCap_partition", transform=transform)
data = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)
classes = dataset.find_classes("/content/MapCap_partition")[0]


newpath = "/content/MapCap_segments"
if os.path.exists(newpath):
  shutil.rmtree(newpath)
os.makedirs(newpath)

imgnum = 0
for image, label in data:
    texts = ["%s"%(classes[label])]
    text = clip.tokenize(texts).to(device)


    R_text, R_image = interpret(model=model, image=image.cuda(), texts=text, device=device)
    batch_size = text.shape[0]

    for i in range(batch_size):
        # From here on, we're trying to segment the heatmap into distinct patches
        segments = segment_heatmap(R_image[i])
        segnum = 0
        for segment in segments:
            segmented_image = torch.tensor(segment[0]) * image.squeeze(0)

            cropped_segmets = False #@param {type:"boolean"}
            segmets_on_black_background = False #@param {type:"boolean"}
            blur_background_of_segment = True #@param {type:"boolean"}


            if cropped_segmets:
              nonzeroes =  torch.where(segmented_image > 0)
              if nonzeroes[1].size() == torch.Size([0]):
                continue
              else:
                top = torch.min(nonzeroes[1]).item()
                left = torch.min(nonzeroes[2]).item()
                max_row = torch.max(nonzeroes[1]).item()
                max_col = torch.max(nonzeroes[2]).item()
                height = max_row - top + 1
                width = max_col - left + 1
                segmented_image = torchvision.transforms.functional.crop(segmented_image, top, left, height, width)

            elif segmets_on_black_background  or weighted_darkness:
              segmented_image = torch.tensor(segment[0]) * image.squeeze(0)

            elif blur_background_of_segment:
              blurred = torchvision.transforms.GaussianBlur(25, sigma=(10.0, 25.0)).forward(image.squeeze(0))
              invert_mask = np.where(segment[0], 0, 1)
              segmented_image = blurred * invert_mask  + segmented_image

            classPath = os.path.join(newpath, classes[label.item()])
            if not os.path.exists(classPath):
                os.makedirs(classPath)
            torchvision.utils.save_image(segmented_image,  os.path.join(classPath, "img%i-seg%i.JPEG"%(imgnum,segnum)))
            segnum += 1
    imgnum += 1


In [ ]:
#@title Import model

import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from typing import Tuple, List, Union, Optional
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from google.colab import files
import skimage.io as io
import PIL.Image
from IPython.display import Image 


N = type(None)
V = np.array
ARRAY = np.ndarray
ARRAYS = Union[Tuple[ARRAY, ...], List[ARRAY]]
VS = Union[Tuple[V, ...], List[V]]
VN = Union[V, N]
VNS = Union[VS, N]
T = torch.Tensor
TS = Union[Tuple[T, ...], List[T]]
TN = Optional[T]
TNS = Union[Tuple[TN, ...], List[TN]]
TSN = Optional[TS]
TA = Union[T, ARRAY]


D = torch.device
CPU = torch.device('cpu')


def get_device(device_id: int) -> D:
    if not torch.cuda.is_available():
        return CPU
    device_id = min(torch.cuda.device_count() - 1, device_id)
    return torch.device(f'cuda:{device_id}')


CUDA = get_device

current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory), "pretrained_models")
os.makedirs(save_path, exist_ok=True)
model_path = os.path.join(save_path, 'model_wieghts.pt')


class MLP(nn.Module):

    def forward(self, x: T) -> T:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) -1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class ClipCaptionModel(nn.Module):

    #@functools.lru_cache #FIXME
    def get_dummy_token(self, batch_size: int, device: D) -> T:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: T, prefix: T, mask: Optional[T] = None, labels: Optional[T] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        #print(embedding_text.size()) #torch.Size([5, 67, 768])
        #print(prefix_projections.size()) #torch.Size([5, 1, 768])
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, prefix_size: int = 512):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        if prefix_length > 10:  # not enough memory
            self.clip_project = nn.Linear(prefix_size, self.gpt_embedding_size * prefix_length)
        else:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2, self.gpt_embedding_size * prefix_length))


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self


def generate_beam(model, tokenizer, beam_size: int = 5, prompt=None, embed=None,
                  entry_length=67, temperature=1., stop_token: str = '.'):

    model.eval()
    stop_token_index = tokenizer.encode(stop_token)[0]
    tokens = None
    scores = None
    device = next(model.parameters()).device
    seq_lengths = torch.ones(beam_size, device=device)
    is_stopped = torch.zeros(beam_size, device=device, dtype=torch.bool)
    with torch.no_grad():
        if embed is not None:
            generated = embed
        else:
            if tokens is None:
                tokens = torch.tensor(tokenizer.encode(prompt))
                tokens = tokens.unsqueeze(0).to(device)
                generated = model.gpt.transformer.wte(tokens)
        for i in range(entry_length):
            outputs = model.gpt(inputs_embeds=generated)
            logits = outputs.logits
            logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
            logits = logits.softmax(-1).log()
            if scores is None:
                scores, next_tokens = logits.topk(beam_size, -1)
                generated = generated.expand(beam_size, *generated.shape[1:])
                next_tokens, scores = next_tokens.permute(1, 0), scores.squeeze(0)
                if tokens is None:
                    tokens = next_tokens
                else:
                    tokens = tokens.expand(beam_size, *tokens.shape[1:])
                    tokens = torch.cat((tokens, next_tokens), dim=1)
            else:
                logits[is_stopped] = -float(np.inf)
                logits[is_stopped, 0] = 0
                scores_sum = scores[:, None] + logits
                seq_lengths[~is_stopped] += 1
                scores_sum_average = scores_sum / seq_lengths[:, None]
                scores_sum_average, next_tokens = scores_sum_average.view(-1).topk(beam_size, -1)
                next_tokens_source = next_tokens // scores_sum.shape[1]
                seq_lengths = seq_lengths[next_tokens_source]
                next_tokens = next_tokens % scores_sum.shape[1]
                next_tokens = next_tokens.unsqueeze(1)
                tokens = tokens[next_tokens_source]
                tokens = torch.cat((tokens, next_tokens), dim=1)
                generated = generated[next_tokens_source]
                scores = scores_sum_average * seq_lengths
                is_stopped = is_stopped[next_tokens_source]
            next_token_embed = model.gpt.transformer.wte(next_tokens.squeeze()).view(generated.shape[0], 1, -1)
            generated = torch.cat((generated, next_token_embed), dim=1)
            is_stopped = is_stopped + next_tokens.eq(stop_token_index).squeeze()
            if is_stopped.all():
                break
    scores = scores / seq_lengths
    output_list = tokens.cpu().numpy()
    output_texts = [tokenizer.decode(output[:int(length)]) for output, length in zip(output_list, seq_lengths)]
    order = scores.argsort(descending=True)
    output_texts = [output_texts[i] for i in order]
    return output_texts


def generate2(
        model,
        tokenizer,
        tokens=None,
        prompt=None,
        embed=None,
        entry_count=1,
        entry_length=67,  # maximum number of words
        top_p=0.8,
        temperature=1.,
        stop_token: str = '.',
):
    model.eval()
    generated_num = 0
    generated_list = []
    stop_token_index = tokenizer.encode(stop_token)[0]
    filter_value = -float("Inf")
    device = next(model.parameters()).device

    with torch.no_grad():

        for entry_idx in trange(entry_count):
            if embed is not None:
                generated = embed
            else:
                if tokens is None:
                    tokens = torch.tensor(tokenizer.encode(prompt))
                    tokens = tokens.unsqueeze(0).to(device)

                generated = model.gpt.transformer.wte(tokens)

            for i in range(entry_length):

                outputs = model.gpt(inputs_embeds=generated)
                logits = outputs.logits
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(nnf.softmax(sorted_logits, dim=-1), dim=-1)
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                                                    ..., :-1
                                                    ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value
                next_token = torch.argmax(logits, -1).unsqueeze(0)
                next_token_embed = model.gpt.transformer.wte(next_token)
                if tokens is None:
                    tokens = next_token
                else:
                    tokens = torch.cat((tokens, next_token), dim=1)
                generated = torch.cat((generated, next_token_embed), dim=1)
                if stop_token_index == next_token.item():
                    break

            output_list = list(tokens.squeeze().cpu().numpy())
            output_text = tokenizer.decode(output_list)
            generated_list.append(output_text)

    return generated_list[0]

In [ ]:
#@title Choose pretrained model - COCO or Coneptual captions


pretrained_model = 'Conceptual captions'  # @param ['COCO', 'Conceptual captions']

if pretrained_model == 'Conceptual captions':
  downloader.download_file("14pXWwB4Zm82rsDdvbGguLfx9F8aM7ovT", model_path)
else:
  downloader.download_file("1IdaBtMSvtyzF0ByVaBHtvM0JYSXRExRX", model_path)

NameError: ignored

In [ ]:
#@title GPU/CPU and load model 

is_gpu = True #@param {type:"boolean"}

device = CUDA(0) if is_gpu else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

prefix_length = 10

model = ClipCaptionModel(prefix_length)

model.load_state_dict(torch.load(model_path, map_location=CPU)) 

model = model.eval() 
device = CUDA(0) if is_gpu else "cpu"
model = model.to(device)

In [ ]:
#@title Label Target directory

newpath = "/content/ClipCap_inferences"
if not os.path.exists(newpath):
  os.makedirs(newpath)
dir = "MapCap_segments"    #@param {type:"string"}
classes = os.listdir("/content/%s"%(dir)) 


In [ ]:
#@title Inference
for cl in classes:
    if cl == ".DS_Store":
        continue
    img_in_class = os.listdir("/content/%s/%s"%(dir,cl))
    #print(cl)
    with open("/content/ClipCap_inferences/%s_results.txt"%(cl), 'w') as f:
        for img in img_in_class:
            use_beam_search = False #@param {type:"boolean"}
            image = io.imread("/content/%s/%s/%s"%(dir,cl,img))
            pil_image = PIL.Image.fromarray(image)
            #pil_img = Image(filename=UPLOADED_FILE)
            #display(pil_image)
            image = preprocess(pil_image).unsqueeze(0).to(device)
   
            with torch.no_grad():
                # if type(model) is ClipCaptionE2E:
                #     prefix_embed = model.forward_image(image)
                # else:
                prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
                prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)
            if use_beam_search:
                generated_text_prefix = generate_beam(model, tokenizer, embed=prefix_embed)[0]
            else:
                generated_text_prefix = generate2(model, tokenizer, embed=prefix_embed)

            f.write(generated_text_prefix)
            f.write("\n")
            #print('\n')
            #print(generated_text_prefix)
        f.close()


In [ ]:
#@title Download Text Files
from google.colab import files
!zip -r /content/Results.zip /content/ClipCap_inferences


  adding: content/To_Upload/ (stored 0%)
  adding: content/To_Upload/cropped_segs/ (stored 0%)
  adding: content/To_Upload/cropped_segs/European fire salamander/ (stored 0%)
  adding: content/To_Upload/cropped_segs/European fire salamander/img1-seg0.JPEG (deflated 18%)
  adding: content/To_Upload/cropped_segs/European fire salamander/img0-seg1.JPEG (deflated 25%)
  adding: content/To_Upload/cropped_segs/European fire salamander/img0-seg0.JPEG (deflated 11%)
  adding: content/To_Upload/segments_on_black_background/ (stored 0%)
  adding: content/To_Upload/segments_on_black_background/European fire salamander/ (stored 0%)
  adding: content/To_Upload/segments_on_black_background/European fire salamander/img1-seg0.JPEG (deflated 53%)
  adding: content/To_Upload/segments_on_black_background/European fire salamander/img0-seg1.JPEG (deflated 59%)
  adding: content/To_Upload/segments_on_black_background/European fire salamander/img0-seg0.JPEG (deflated 38%)
  adding: content/To_Upload/.ipynb_ch

## Summarize Generated Text files

In [ ]:
import os
os.chdir("../CS1430-MapCap/code/mi-unsup-summ")

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import get_matrices
import numpy as np

root_path = "../../CC_inferences/heatmap_mask_no_segmenting"
matrix_dir = "./matrix_blur"
os.makedirs(matrix_dir, exist_ok=True)

classes_of_interest = ["tabby", "plunger", "tarantula", "Egyptian cat", "beer bottle", "European fire salamander"]
for f in os.listdir(root_path):
  print(f)
  if np.any([cl in f for cl in classes_of_interest]):
    get_matrices.main(os.path.join(root_path, f), os.path.join(matrix_dir, f + "_hmap_matrix.pkl"), "cuda")

neck brace_results.txt
lemon_results.txt
tarantula_results.txt


Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
4k abstract of a flower. a snail crawls on a black background.
tensor([[50256,   604,    74, 12531,   286,   257, 15061,    13,   257,  4268,
           286,  1660,  7348,   832,   262,  1633,    13, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256,   604,    74, 12531,   286,   257, 15061,    13,   257,  6016,
           290,  6016, 27223,  3568,   287,   262, 11854,    13, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256,   604,    74, 12531,   286,   257, 15061,    13,   262,  8824,
           287,   262,  1755,  6766,    13, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256,   604,    74, 12531,   286,   257, 15061,    13,   604,    74,
          1657, 17316,  9640,   329,  1180,  2995,   290,  4493, 43179, 50256,
         50256, 50256, 50256, 50256, 50256],
        [50256,   604,    74, 12531,   286,   257, 15061,    13,  5916, 12

In [ ]:
import get_interpolated_output

outdir = "./summaries_hmap"
os.makedirs(outdir, exist_ok=True)

for f in os.listdir(root_path):
  print(f)
  if np.any([cl in f for cl in classes_of_interest]):
    get_interpolated_output.main(os.path.join(outdir, f), os.path.join(root_path, f), os.path.join(matrix_dir, f + "_hmap_matrix.pkl"))

neck brace_results.txt
lemon_results.txt
tarantula_results.txt
['a piece of pizza rotates on black background.', "a dolly shot of a woman's face as she looks at a red heart on a white background.", 'the moon and earth seen in the night sky.', 'a lot of hearts in the darkness.', '3d animation of a planet in deep space.', 'flight of gold coins falling on a black background.', 'fire flames on a black background.', 'a dolly shot of a tree branch rotating on a black background.', 'moon in the night sky.', '4k light leaks footage for different events and projects!.', '4k light leaks footage for different events and projects!.', 'falling snow on a black background.', 'the planet in the universe.', 'a sphere made of white marble rotates on black background.', 'moon in the night sky.', 'the earth rotates in black space.', 'a hand with a hammer hammer hitting a nail into a wall.', 'a glowing ball of light rotates on a black background.', 'the moon in the dark sky.', 'the spacecraft flies through

In [ ]:
!zip -r summaries_heatmap.zip summaries_hmap
print(os.listdir("."))

  adding: summaries_hmap/ (stored 0%)
  adding: summaries_hmap/tabby_results.txt9 (deflated 51%)
  adding: summaries_hmap/Egyptian cat_results.txt3 (deflated 45%)
  adding: summaries_hmap/beer bottle_results.txt4 (deflated 50%)
  adding: summaries_hmap/Egyptian cat_results.txt8 (deflated 54%)
  adding: summaries_hmap/tarantula_results.txt8 (deflated 51%)
  adding: summaries_hmap/plunger_results.txt5 (deflated 48%)
  adding: summaries_hmap/tarantula_results.txt6 (deflated 44%)
  adding: summaries_hmap/plunger_results.txt6 (deflated 51%)
  adding: summaries_hmap/European fire salamander_results.txt4 (deflated 54%)
  adding: summaries_hmap/plunger_results.txt7 (deflated 52%)
  adding: summaries_hmap/Egyptian cat_results.txt7 (deflated 55%)
  adding: summaries_hmap/tabby_results.txt5 (deflated 49%)
  adding: summaries_hmap/European fire salamander_results.txt6 (deflated 56%)
  adding: summaries_hmap/plunger_results.txt9 (deflated 51%)
  adding: summaries_hmap/European fire salamander_resul